In [ ]:
# 必要なライブラリをインポート
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit.visualization import plot_distribution
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeBrisbane
import numpy as np
from itertools import product
from fractions import Fraction
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline

In [4]:
# ノイズなしシミュレーター
def run_qc_on_simulator(qc, backend=AerSimulator(), shots=1024):
    compiled_circuit = transpile(qc, backend, optimization_level=1)
    sampler = Sampler(mode=backend)
    job = sampler.run([compiled_circuit], shots=shots)
    result = job.result()
    counts = result[0].data.c.get_counts()
    return counts

# ノイズありシミュレーター
def run_qc_on_noised_simulator(qc, backend=FakeBrisbane(), shots=1024):
    noise_model = NoiseModel.from_backend(backend)
    simulator = AerSimulator(noise_model=noise_model)
    compiled_circuit = transpile(qc, simulator, optimization_level=1)
    sampler = Sampler(mode=simulator)
    job = sampler.run([compiled_circuit], shots=shots)
    result = job.result()
    counts = result[0].data.c.get_counts()
    return counts

In [ ]:
# N = 15, a = 7 の場合のShorのアルゴリズムの本体
def c_amod15(a, l):
    if a not in [2, 4, 7, 8, 11, 13]:
        raise ValueError("a must be one of the following values: 2, 4, 7, 8, 11, or 13.")
    U = QuantumCircuit(4)
    if a in [2, 13]:
        U.swap(3, 2)
        U.swap(2, 1)
        U.swap(1, 0)
    elif a in [4, 11]:
        U.swap(3, 1)
        U.swap(2, 0)
    elif a in [7, 8]:
        U.swap(1, 0)
        U.swap(2, 1)
        U.swap(3, 2)
    if a in [7, 11, 13]:
        U.x([0, 1, 2, 3])
    
    U_power = U.repeat(2 ** l)
    gate = U_power.to_gate()
    gate.name = f"{a}^{2 ** l} mod 15"
    c_gate = gate.control()
    
    return c_gate

def qft_dagger(qreg):
    qc = QuantumCircuit(qreg)

    for j in range(qreg.size // 2):
        qc.swap(qreg[j], qreg[-1-j])
    for itarg in range(qreg.size):
        for ictrl in range(itarg):
            power = ictrl - itarg - 1
            qc.cp(-2. * np.pi * (2 ** power), ictrl, itarg)
        
        qc.h(itarg)
    
    qc.name = "QFT^dagger"
    return qc

def continued_fraction_expansion(sorted_counts):
    print('Register Output    Phase      Count   Probability')
    print('------------------------------------------------')

    measured_phases = []
    for output, count in sorted_counts:
        count = int(count)
        decimal_output = int(output, 2)
        phase = decimal_output / (2 ** n_meas)
        probability = count / shots
        measured_phases.append((phase, count))
        print(f"{output:>8s} ({decimal_output:>3d})  {phase:>6.3f}     {count:>5d}   {probability:>7.3f}")

    print('\nPhase Fraction Analysis')
    print('Phase      Fraction    Possible r    Count')
    print('------------------------------------------')

    # 周期候補の分析
    period_candidates = {}
    for output, count in sorted_counts:
        count = int(count)
        decimal_output = int(output, 2)
        print(f"Output: {output}, Decimal: {decimal_output}, Count: {count}")
        phase = decimal_output / (2 ** n_meas)
        frac = Fraction(phase).limit_denominator(15)
        
        if frac.denominator > 1 and frac.denominator < 15:
            r = frac.denominator
            if r in period_candidates:
                period_candidates[r] += count
            else:
                period_candidates[r] = count
            print(f"{phase:>6.3f}    {frac.numerator:>2d}/{frac.denominator:<2d}        {r:>2d}        {count:>5d}")

    # 最も可能性の高い周期を選択
    if period_candidates:
        sorted_periods = sorted(period_candidates.items(), key=lambda x: x[1], reverse=True)
        most_likely_r = sorted_periods[0][0]
        
        print(f'\nPeriod Estimation (sorted by frequency):')
        print('----------------------------------------')
        for r, total_count in sorted_periods:
            probability = total_count / shots
            print(f"r = {r:>2d}    Count: {total_count:>5d}    Probability: {probability:>6.3f}")
        
        print(f'\nMost likely period: r = {most_likely_r}')
        
        # 因数分解の実行
        if most_likely_r % 2 == 0:
            factor1 = np.gcd(a**(most_likely_r//2) - 1, N)
            factor2 = np.gcd(a**(most_likely_r//2) + 1, N)
            
            print(f'\nFactorization attempt with r = {most_likely_r}:')
            print(f'gcd({a}^{most_likely_r//2} - 1, {N}) = {factor1}')
            print(f'gcd({a}^{most_likely_r//2} + 1, {N}) = {factor2}')
            
            if factor1 > 1 and factor1 < N:
                print(f'\nSuccess! {N} = {factor1} × {N // factor1}')
            elif factor2 > 1 and factor2 < N:
                print(f'\nSuccess! {N} = {factor2} × {N // factor2}')
            else:
                print(f'\nNo non-trivial factors found')
        else:
            print(f'\nPeriod r = {most_likely_r} is odd, retry needed')
    else:
        print('\nNo valid period candidates found')

    # 結果の可視化
    top5_counts = dict(sorted_counts)
    plot_distribution(top5_counts, title=f"Shor's Algorithm Results (N={N}, a={a})")
    plt.show()


# N = 15, a = 7 の場合のShorのアルゴリズムを実行
N = 15
a = 7

n_meas = 8
qreg_meas = QuantumRegister(n_meas, name="meas")
qreg_aux = QuantumRegister(4, name="aux")
creg_meas = ClassicalRegister(n_meas, name="c")

qc = QuantumCircuit(qreg_meas, qreg_aux, creg_meas)
qc.h(qreg_meas)
qc.x(qreg_aux[0])
qc.barrier()

for l, ctrl in enumerate(qreg_meas):
    qc.append(c_amod15(a, l), qargs=([ctrl] + qreg_aux[:]))

qc.barrier()
qc.append(qft_dagger(qreg_meas), qargs=qreg_meas)

qc.barrier()
qc.measure(qreg_meas, creg_meas)

# 測定結果を頻度順にソート
shots = 1024
counts = run_qc_on_simulator(qc, shots=shots)
# counts = run_qc_on_noised_simulator(qc, shots=shots)

sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

continued_fraction_expansion(sorted_counts)

In [ ]:
# Measurement Error Mitigationのための関数群
def collect_noisy_counts(n):
    noisy_counts = []
    bitstrings = [''.join(bits) for bits in product('01', repeat=n)]

    for state in bitstrings:
        qc = QuantumCircuit(n, n)

        for i, bit in enumerate(reversed(state)):
            if bit == '1':
                qc.x(i)

        qc.measure(range(n), range(n))
        
        counts = run_qc_on_noised_simulator(qc)
        noisy_counts.append(counts)
        print(f"{state} becomes {counts}")

    return noisy_counts


def make_matrix(noisy_counts):
    all_bitstrings = sorted({bit for counts in noisy_counts for bit in counts})
    n = len(all_bitstrings)
    
    matrix = np.zeros((n, n))
    
    for i, counts in enumerate(noisy_counts[:n]):
        total = sum(counts.values())
        for j, bit in enumerate(all_bitstrings):
            matrix[i, j] = counts.get(bit, 0) / total if total > 0 else 0.0
            
    return matrix, all_bitstrings


def correct_counts(raw_counts, transition_matrix, bit_order):
    y = np.array([raw_counts.get(b, 0) for b in bit_order], dtype=float)
    
    try:
        A_inv = np.linalg.inv(transition_matrix)
    except np.linalg.LinAlgError:
        raise ValueError("遷移行列が特異（逆行列が存在しません）")
    
    x = np.dot(A_inv, y)
    x = np.clip(x, 0, None)
    
    return {bit: x[i] for i, bit in enumerate(bit_order)}

In [ ]:
# Measurement Error Mitigationの実行
noisy_counts = collect_noisy_counts(8)
M = make_matrix(noisy_counts)
print(M)
noised_aer_counts_mitigated = correct_counts(counts, M[0], M[1])

In [ ]:
noised_aer_counts_mitigated = correct_counts(counts, M[0], M[1])
top5_counts = sorted(noised_aer_counts_mitigated.items(), key=lambda x: x[1], reverse=True)[:5]
continued_fraction_expansion(top5_counts)

In [ ]:
# ショアのアルゴリズム用 ZNE関数群
def fold_circuit(circuit, scale_factor):
    if scale_factor % 2 == 0 or scale_factor < 1:
        raise ValueError("scale_factor must be an odd integer >= 1")
    
    folded = QuantumCircuit(circuit.num_qubits, circuit.num_clbits)

    qubit_map = {q: folded.qubits[i] for i, q in enumerate(circuit.qubits)}
    clbit_map = {c: folded.clbits[i] for i, c in enumerate(circuit.clbits)}

    for instr, qargs, cargs in circuit.data:
        new_qargs = [qubit_map[q] for q in qargs]
        new_cargs = [clbit_map[c] for c in cargs]

        if instr.name == "measure":
            folded.append(instr, new_qargs, new_cargs)
        else:
            for _ in range((scale_factor - 1) // 2):
                folded.append(instr, new_qargs, new_cargs)
                if instr.inverse() is not None:
                    folded.append(instr.inverse(), new_qargs, new_cargs)
            folded.append(instr, new_qargs, new_cargs)
    
    return folded

def normalize_counts(counts):
    total = sum(counts.values())
    return {k: v / total for k, v, in counts.items()}

def zne_prob_extrapolation(circuit, scale_factors=[1,3,5]):
    all_probs = defaultdict(list)
    all_bitstrings = set()

    for scale in scale_factors:
        folded = fold_circuit(circuit, scale)
        folded.measure_all()
        # counts = run_qc_on_simulator(folded)
        counts = run_qc_on_noised_simulator(folded)
        probs = normalize_counts(counts)

        for bitstring in probs:
            all_bitstrings.add(bitstring)
        
        for b in all_bitstrings:
            all_probs[b].append(probs.get(b, 0.0))
    
    extrapolated_probs = {}
    for b in all_bitstrings:
        y = all_probs[b]
        x = scale_factors[:len(y)]
        coeffs = np.polyfit(x, y, deg=1)
        extrapolated_probs[b] = max(0.0, np.polyval(coeffs, 0))
    
    total = sum(extrapolated_probs.values())
    for b in extrapolated_probs:
        extrapolated_probs[b] /= total

    top_items = sorted(extrapolated_probs.items(), key=lambda x: x[1], reverse=True)[:5]
    for bitstring, prob in top_items:
        print(f"{bitstring}: {prob:.4f}")
    
    plt.bar(*zip(*top_items))
    plt.ylabel("Extrapolated Probability")
    plt.xlabel("Bitstring")
    plt.title("ZNE-Extrapolated Probabilities")
    plt.show()

    return extrapolated_probs

N = 15
a = 7

n_meas = 8
qreg_meas = QuantumRegister(n_meas, name="meas")
qreg_aux = QuantumRegister(4, name="aux")
creg_meas = ClassicalRegister(n_meas, name="out")

qc = QuantumCircuit(qreg_meas, qreg_aux, creg_meas)
qc.h(qreg_meas)
qc.x(qreg_aux[0])
qc.barrier()

for l, ctrl in enumerate(qreg_meas):
    qc.append(c_amod15(a, l), qargs=([ctrl] + qreg_aux[:]))

qc.barrier()
qc.append(qft_dagger(qreg_meas), qargs=qreg_meas)

qc.barrier()
qc.measure(qreg_meas, creg_meas)

extrapolated_probs = zne_prob_extrapolation(qc)

# 位相推定と因数分解の実行
print('\nPhase Fraction Analysis')
print('Phase      Fraction    Possible r    Probability')
print('---------------------------------------------')

shots = 1024  # 確率を計算するための仮想ショット数
period_candidates = {}

for bitstring, prob in extrapolated_probs.items():
    decimal_output = int(bitstring, 2)
    phase = decimal_output / (2 ** n_meas)
    frac = Fraction(phase).limit_denominator(15)
    
    if frac.denominator > 1 and frac.denominator < 15:
        r = frac.denominator
        if r in period_candidates:
            period_candidates[r] += prob
        else:
            period_candidates[r] = prob
        print(f"{phase:>6.3f}    {frac.numerator:>2d}/{frac.denominator:<2d}        {r:>2d}        {prob:>7.4f}")

# 最も可能性の高い周期を選択
if period_candidates:
    sorted_periods = sorted(period_candidates.items(), key=lambda x: x[1], reverse=True)
    most_likely_r = sorted_periods[0][0]
    
    print(f'\nPeriod Estimation (sorted by probability):')
    print('------------------------------------------')
    for r, total_prob in sorted_periods:
        print(f"r = {r:>2d}    Probability: {total_prob:>7.4f}")
    
    print(f'\nMost likely period: r = {most_likely_r}')
    
    # 因数分解の実行
    if most_likely_r % 2 == 0:
        factor1 = np.gcd(a**(most_likely_r//2) - 1, N)
        factor2 = np.gcd(a**(most_likely_r//2) + 1, N)
        
        print(f'\nFactorization attempt with r = {most_likely_r}:')
        print(f'gcd({a}^{most_likely_r//2} - 1, {N}) = {factor1}')
        print(f'gcd({a}^{most_likely_r//2} + 1, {N}) = {factor2}')
        
        if factor1 > 1 and factor1 < N:
            print(f'\nSuccess! {N} = {factor1} × {N // factor1}')
        elif factor2 > 1 and factor2 < N:
            print(f'\nSuccess! {N} = {factor2} × {N // factor2}')
        else:
            print(f'\nNo non-trivial factors found')
    else:
        print(f'\nPeriod r = {most_likely_r} is odd, retry needed')
else:
    print('\nNo valid period candidates found')

In [ ]:
# ショアのアルゴリズム Measurement Error MitigationとZNEの同時実行
def fold_circuit(circuit, scale_factor):
    if scale_factor % 2 == 0 or scale_factor < 1:
        raise ValueError("scale_factor must be an odd integer >= 1")
    
    folded = QuantumCircuit(circuit.num_qubits, circuit.num_clbits)

    qubit_map = {q: folded.qubits[i] for i, q in enumerate(circuit.qubits)}
    clbit_map = {c: folded.clbits[i] for i, c in enumerate(circuit.clbits)}

    for instr, qargs, cargs in circuit.data:
        new_qargs = [qubit_map[q] for q in qargs]
        new_cargs = [clbit_map[c] for c in cargs]

        if instr.name == "measure":
            folded.append(instr, new_qargs, new_cargs)
        else:
            for _ in range((scale_factor - 1) // 2):
                folded.append(instr, new_qargs, new_cargs)
                if instr.inverse() is not None:
                    folded.append(instr.inverse(), new_qargs, new_cargs)
            folded.append(instr, new_qargs, new_cargs)
    
    return folded

def normalize_counts(counts):
    total = sum(counts.values())
    return {k: v / total for k, v, in counts.items()}

def zne_prob_extrapolation(circuit, scale_factors=[1,3,5]):
    all_probs = defaultdict(list)
    all_bitstrings = set()

    for scale in scale_factors:
        folded = fold_circuit(circuit, scale)
        folded.measure_all()
        counts = run_qc_on_simulator(folded)
        # counts = run_qc_on_noised_simulator(folded)
        counts = correct_counts(counts, M[0], M[1])
        probs = normalize_counts(counts)

        for bitstring in probs:
            all_bitstrings.add(bitstring)
        
        for b in all_bitstrings:
            all_probs[b].append(probs.get(b, 0.0))
    
    extrapolated_probs = {}
    for b in all_bitstrings:
        y = all_probs[b]
        x = scale_factors[:len(y)]
        coeffs = np.polyfit(x, y, deg=1)
        extrapolated_probs[b] = max(0.0, np.polyval(coeffs, 0))
    
    total = sum(extrapolated_probs.values())
    for b in extrapolated_probs:
        extrapolated_probs[b] /= total

    top_items = sorted(extrapolated_probs.items(), key=lambda x: x[1], reverse=True)[:5]
    for bitstring, prob in top_items:
        print(f"{bitstring}: {prob:.4f}")
    
    plt.bar(*zip(*top_items))
    plt.ylabel("Extrapolated Probability")
    plt.xlabel("Bitstring")
    plt.title("Measurement Error Mitigation & ZNE Probabilities (Top 5)")
    # plt.grid(True)
    plt.show()

    return extrapolated_probs

N = 15
a = 7

n_meas = 8
qreg_meas = QuantumRegister(n_meas, name="meas")
qreg_aux = QuantumRegister(4, name="aux")
creg_meas = ClassicalRegister(n_meas, name="out")

qc = QuantumCircuit(qreg_meas, qreg_aux, creg_meas)
qc.h(qreg_meas)
qc.x(qreg_aux[0])
qc.barrier()

for l, ctrl in enumerate(qreg_meas):
    qc.append(c_amod15(a, l), qargs=([ctrl] + qreg_aux[:]))

qc.barrier()
qc.append(qft_dagger(qreg_meas), qargs=qreg_meas)

qc.barrier()
qc.measure(qreg_meas, creg_meas)

extrapolated_probs = zne_prob_extrapolation(qc)

# 位相推定と因数分解の実行
print('\nPhase Fraction Analysis')
print('Phase      Fraction    Possible r    Probability')
print('---------------------------------------------')

shots = 1024  # 確率を計算するための仮想ショット数
period_candidates = {}

for bitstring, prob in extrapolated_probs.items():
    decimal_output = int(bitstring, 2)
    phase = decimal_output / (2 ** n_meas)
    frac = Fraction(phase).limit_denominator(15)
    
    if frac.denominator > 1 and frac.denominator < 15:
        r = frac.denominator
        if r in period_candidates:
            period_candidates[r] += prob
        else:
            period_candidates[r] = prob
        print(f"{phase:>6.3f}    {frac.numerator:>2d}/{frac.denominator:<2d}        {r:>2d}        {prob:>7.4f}")

# 最も可能性の高い周期を選択
if period_candidates:
    sorted_periods = sorted(period_candidates.items(), key=lambda x: x[1], reverse=True)
    most_likely_r = sorted_periods[0][0]
    
    print(f'\nPeriod Estimation (sorted by probability):')
    print('------------------------------------------')
    for r, total_prob in sorted_periods:
        print(f"r = {r:>2d}    Probability: {total_prob:>7.4f}")
    
    print(f'\nMost likely period: r = {most_likely_r}')
    
    # 因数分解の実行
    if most_likely_r % 2 == 0:
        factor1 = np.gcd(a**(most_likely_r//2) - 1, N)
        factor2 = np.gcd(a**(most_likely_r//2) + 1, N)
        
        print(f'\nFactorization attempt with r = {most_likely_r}:')
        print(f'gcd({a}^{most_likely_r//2} - 1, {N}) = {factor1}')
        print(f'gcd({a}^{most_likely_r//2} + 1, {N}) = {factor2}')
        
        if factor1 > 1 and factor1 < N:
            print(f'\nSuccess! {N} = {factor1} × {N // factor1}')
        elif factor2 > 1 and factor2 < N:
            print(f'\nSuccess! {N} = {factor2} × {N // factor2}')
        else:
            print(f'\nNo non-trivial factors found')
    else:
        print(f'\nPeriod r = {most_likely_r} is odd, retry needed')
else:
    print('\nNo valid period candidates found')